<a href="https://colab.research.google.com/github/gummadhav/Let_us_Learn/blob/main/Learn_OpenCV_OpenCL_On_CoLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
echo "Updating package lists and installing build essentials..."
apt-get update
apt-get install -y build-essential # Compilers and build tools

echo "Installing OpenCV C++ development libraries..."
apt-get install -y libopencv-dev python3-opencv

echo "Verifying OpenCL development files (usually part of CUDA or standalone dev packages)..."
# Check if OpenCL headers are in a common location
ls -l /usr/include/CL/cl.h

# If the above 'ls' command returns "No such file or directory",
# you might need to install a specific OpenCL header package:
# For example, on some systems, it's 'opencl-headers' or 'libopencl-dev'
# apt-get install -y opencl-headers

# In Colab, the NVIDIA CUDA Toolkit usually brings in OpenCL support.
# We can check for a system-wide OpenCL library
ldconfig -p | grep libOpenCL

Updating package lists and installing build essentials...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,773 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease

In [ ]:
%%writefile opencv_opencl_example.cpp

#include <opencv2/opencv.hpp>
#include <opencv2/core/ocl.hpp>
#include <iostream>
#include <chrono>

int main() {
    if (cv::ocl::haveOpenCL()) {
        std::cout << "OpenCL is available! Trying to enable it for OpenCV.\n";

        if (cv::ocl::useOpenCL()) {
            std::cout << "OpenCL enabled for OpenCV.\n";

            // Corrected way to get platform and device info for OpenCV 4.x
            cv::ocl::Context context;
            context.create(cv::ocl::Device::TYPE_GPU);

            // Get the platform name from the first device
            cv::ocl::Device device = context.device(0);
            //std::cout << "OpenCL Platform: " << device.platformName() << std::endl;
            std::cout << "OpenCL Device Info:\n";
            std::cout << "  Name    : " << device.name() << std::endl;
            std::cout << "  Vendor  : " << device.vendorName() << std::endl;
            std::cout << "  Version : " << device.version() << std::endl;

            // Iterate and print all devices
            for (int i = 0; i < context.ndevices(); ++i) {
                cv::ocl::Device d = context.device(i);
                std::cout << "  Device " << i << ": " << d.name() << " (" << d.type() << ")" << std::endl;
            }
        } else {
            std::cout << "Failed to enable OpenCL for OpenCV. Falling back to CPU.\n";
        }
    } else {
        std::cout << "OpenCL is NOT available. Falling back to CPU.\n";
    }

    std::string input_filename = "test_image.jpg";
    cv::Mat image = cv::imread(input_filename, cv::IMREAD_COLOR);

    if (image.empty()) {
        std::cerr << "Error: Could not open or find the image " << input_filename << std::endl;
        std::cerr << "Please upload 'test_image.jpg' to the Colab environment." << std::endl;
        return -1;
    }
    std::cout << "Image '" << input_filename << "' loaded. Dimensions: " << image.cols << "x" << image.rows << std::endl;

    cv::UMat u_image, u_grayscale;
    image.copyTo(u_image);

    auto start = std::chrono::high_resolution_clock::now();
    cv::cvtColor(u_image, u_grayscale, cv::COLOR_BGR2GRAY);
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> duration_ms = end - start;

    std::cout << "Grayscale conversion took: " << duration_ms.count() << " ms\n";

    std::string output_filename = "test_image_grayscale.jpg";
    bool success = cv::imwrite(output_filename, u_grayscale.getMat(cv::ACCESS_READ));

    if (success) {
        std::cout << "Grayscale image saved as '" << output_filename << "'" << std::endl;
    } else {
        std::cerr << "Error: Could not save the grayscale image." << std::endl;
        return -1;
    }

    return 0;
}

Overwriting opencv_opencl_example.cpp


In [ ]:
%%shell
# Compile the C++ code, linking with OpenCV libraries
g++ opencv_opencl_example.cpp -o opencv_opencl_app $(pkg-config --cflags --libs opencv4)

# Check if compilation was successful
if [ -f opencv_opencl_app ]; then
    echo "Compilation successful. Executable 'opencv_opencl_app' created."
else
    echo "Compilation FAILED. Check for errors above."
fi

Compilation successful. Executable 'opencv_opencl_app' created.


In [ ]:
%%shell
ls -l opencv_opencl_app

-rwxr-xr-x 1 root root 37600 Aug  3 17:17 opencv_opencl_app


In [ ]:
%%shell
./opencv_opencl_app

OpenCL is available! Trying to enable it for OpenCV.
OpenCL enabled for OpenCV.
OpenCL Device Info:
  Name    : Tesla T4
  Vendor  : NVIDIA Corporation
  Version : OpenCL 3.0 CUDA
  Device 0: Tesla T4 (4)
Image 'test_image.jpg' loaded. Dimensions: 382x509
Grayscale conversion took: 552.237 ms
Grayscale image saved as 'test_image_grayscale.jpg'


In [ ]:
%%writefile invert_image.cpp

#include <opencv2/opencv.hpp>
#include <opencv2/core/ocl.hpp>
#include <iostream>

const char* opencl_kernel_code = R"(
__kernel void invert_image(__global uchar* src, __global uchar* dst, int total) {
    int idx = get_global_id(0);
    if (idx < total) {
        dst[idx] = 255 - src[idx];
    }
}
)";

int main() {
    // Check for OpenCL availability
    if (!cv::ocl::haveOpenCL()) {
        std::cerr << "OpenCL is not available.\n";
        return -1;
    }

    cv::ocl::Context context;
    if (!context.create(cv::ocl::Device::TYPE_GPU)) {
        std::cerr << "Failed to create OpenCL context.\n";
        return -1;
    }

    cv::ocl::Device device = context.device(0);
    std::cout << "Running on device: " << device.name() << "\n";

    // Load grayscale image
    cv::Mat img = cv::imread("test_image.jpg", cv::IMREAD_GRAYSCALE);
    if (img.empty()) {
        std::cerr << "Image load failed.\n";
        return -1;
    }

    // Upload image to UMat (automatically uses OpenCL if available)
    cv::UMat u_src, u_dst;
    img.copyTo(u_src);
    u_dst.create(u_src.size(), u_src.type());

    // Create OpenCL kernel from source
    cv::ocl::ProgramSource source(opencl_kernel_code);
    cv::String errmsg;
    cv::ocl::Program program = context.getProg(source, "", errmsg);
    if (!errmsg.empty()) std::cerr << "Kernel compile error: " << errmsg << "\n";

    cv::ocl::Kernel kernel("invert_image", program);
    if (kernel.empty()) {
        std::cerr << "Failed to create OpenCL kernel.\n";
        return -1;
    }

    size_t total = u_src.total();
    kernel.set(0, cv::ocl::KernelArg::ReadOnly(u_src));
    kernel.set(1, cv::ocl::KernelArg::WriteOnly(u_dst));
    kernel.set(2, (int)total);

    // Run the kernel
    size_t globalSize[] = { total };
    if (!kernel.run(1, globalSize, NULL, true)) {
        std::cerr << "OpenCL kernel execution failed.\n";
        return -1;
    }

    // Save result
    cv::Mat result;
    u_dst.copyTo(result);
    cv::imwrite("inverted_image.jpg", result);
    std::cout << "Inverted image saved as 'inverted_image.jpg'\n";

    return 0;
}


Writing invert_image.cpp


In [29]:
%%shell
# Compile the C++ code, linking with OpenCV libraries
g++ invert_image.cpp -o invert_image $(pkg-config --cflags --libs opencv4)

# Check if compilation was successful
if [ -f invert_image ]; then
    echo "Compilation successful. Executable 'opencv_opencl_app' created."
else
    echo "Compilation FAILED. Check for errors above."
fi

Compilation successful. Executable 'opencv_opencl_app' created.


In [30]:
%%shell

./invert_image

Running on device: Tesla T4
Inverted image saved as 'inverted_image.jpg'
